In [1]:
import numpy as np

In [2]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   5.08 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   5.77 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   9.34 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.07 MB/s
vincent-0.4.4- 100% |###################

In [3]:

table_from_top = 1
wikipedia_page = 'List of postal codes of Canada: M'
trace = False



In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Use SHIFT+TAB keys to popup inplace code help
%config IPCompleter.greedy = True

# Output multiple statements from one input cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:

wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'.format(wikipedia_page)
page = requests.get(wikipedia_url)
soup = BeautifulSoup(page.content, 'lxml')
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[table_from_top - 1]



In [6]:
feature_names = []

header_row = table.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))
    feature_name.replace('\n', '')
    feature_names.append(feature_name)

'Postcode'

'Borough'

'Neighbourhood'

In [7]:
def has_coords(tag):
    if tag.has_attr('class'):
        if tag['class'][0] == 'latitude' or tag['class'][0] == 'longitude':
            return True
    return False

def get_coords(child):
    coords = []
    for coord in child.find_all(has_coords):
        coords.append(coord.string)
    if coords:
        if trace:
            return 'C = {}'.format(' '.join(coords))
        else:
            return ' '.join(coords)
    else:
        return ''

samples = []
sample_rows = table.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.string
        if text:
            if trace:
                features.append('T = {}'.format(text))
            else:
                features.append(text)
            continue
        
        for child in feature_col.children:
            if child.name == 'span':
                if child.has_attr('class'):
                    if child['class'] == 'display:none':
                        continue
                if child.find_all(has_coords):
                    feature_value = get_coords(child)
                    if feature_value:
                        break
                    else:
                        continue
            if child.name == 'sup':
                continue
            if child.name == 'a':
                if child.string[0] == '[':
                    continue            
            if child.name == 'a':
                if trace:
                    feature_value = 'A = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            if child.name == 'font':
                if trace:
                    feature_value = 'F = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            try:
                # feature_value = '' for any tags not covered above
                content = child.contents
            except AttributeError:
                # Handle whitespace between child tags, treated as a child string
                if child.isspace():
                    continue
                if trace:
                    feature_value = 'E = {}'.format(child)
                else:
                    feature_value = child
                break
        features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))

In [8]:
df = pd.DataFrame(samples)
df.head()
df.tail()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


,Borough,Neighbourhood,Postcode
284,Etobicoke,Mimico NW,M8Z
285,Etobicoke,The Queensway West,M8Z
286,Etobicoke,Royal York South West,M8Z
287,Etobicoke,South of Bloor,M8Z
288,Not assigned,Not assigned,M9Z


In [9]:
dataset_file_name = 'List_of_postal_codes_of_Canada:_M.cvs'.format(wikipedia_page)
if not trace:
    df.to_csv(dataset_file_name, index=False)

Drop rows with value "Not assigned" in Borough column

In [10]:

df[df.Borough != 'Not assigned']

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
10,Etobicoke,Islington Avenue,M9A
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [11]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
df2 = df2.rename(columns={'Postal Code': 'Postcode'})
df2.head(3)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [13]:
df_toronto=pd.merge(df, df2, on='Postcode')
df_toronto

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,North York,Lawrence Heights,M6A,43.718518,-79.464763
5,North York,Lawrence Manor,M6A,43.718518,-79.464763
6,Queen's Park,Not assigned,M7A,43.662301,-79.389494
7,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
8,Scarborough,Rouge,M1B,43.806686,-79.194353
9,Scarborough,Malvern,M1B,43.806686,-79.194353


Create a Map of Toronto City (with its Postal Codes' Regions)

In [14]:
df_toronto.head(10)

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,North York,Lawrence Heights,M6A,43.718518,-79.464763
5,North York,Lawrence Manor,M6A,43.718518,-79.464763
6,Queen's Park,Not assigned,M7A,43.662301,-79.389494
7,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
8,Scarborough,Rouge,M1B,43.806686,-79.194353
9,Scarborough,Malvern,M1B,43.806686,-79.194353


In [15]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)
    

map_toronto


In [16]:
df_toronto.to_csv("df_toronto.csv", index=False, encoding='utf8')

In [17]:
list(df_toronto)

['Borough', 'Neighbourhood\n', 'Postcode', 'Latitude', 'Longitude']

In [18]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood\n']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
map_toronto

In [20]:
# df_toronto['Borough'] == 'Downtown Toronto'

# selecting only neighborhoods regarding to "Scarborough" borough.
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto']
downtown_data.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
2,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
13,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
14,Downtown Toronto,Garden District,M5B,43.657162,-79.378937
27,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418


In [21]:
address_scar = 'Downtown Toronto, Toronto'
latitude_scar = 43.654260
longitude_scar = -79.360636
print('The geograpical coordinate of "Downtown Toronto" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Downtown = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood\n']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Downtown)  
    
map_Downtown

The geograpical coordinate of "Downtown Toronto" are: 43.65426, -79.360636.


In [22]:
map_Downtown

In [23]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [24]:
# @hiddel_cell
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [25]:
donwtown_data = df_toronto[df_toronto['Borough'] == 'Fowntown'].reset_index(drop=True)
downtown_data.head(7)

,Borough,Neighbourhood,Postcode,Latitude,Longitude
2,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
13,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
14,Downtown Toronto,Garden District,M5B,43.657162,-79.378937
27,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
37,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
41,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383


Create Map of Downtown Toronto

In [26]:
address_scar = 'Downtown,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Downtown are {}, {}.'.format(latitude_scar, longitude_scar))



The geograpical coordinate of Downtown are 43.773077, -79.257774.


In [27]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood\n']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb


Get the top 100 venues in all neighborhoods in Downtown


In [35]:
print('Crawling different neighborhoods inside "downtown"')
downtown_foursquare_dataset = foursquare_crawler(list(downtown_data['Postcode']),
                                                   list(downtown_data['Neighbourhood\n']),
                                                   list(downtown_data['Latitude']),
                                                   list(downtown_data['Longitude']),)


Crawling different neighborhoods inside "downtown"
1.
Data is Obtained, for the Postal Code M5A (and Neighborhoods Harbourfront) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M5A (and Neighborhoods Regent Park) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M5B (and Neighborhoods Ryerson) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M5B (and Neighborhoods Garden District
) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M5C (and Neighborhoods St. James Town) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M5E (and Neighborhoods Berczy Park) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M5G (and Neighborhoods Central Bay Street
) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M6G (and Neighborhoods Christie
) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M5H (and Neighborhoods Adelaide) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Code M5H (and Neighborhoods King) SUCCESSFULLY.
11.
Data is Obtained, for the Postal 

In [36]:
import pickle
with open("downtown_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(downtown_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')


RecursionError: maximum recursion depth exceeded

In [37]:


with open("downtown_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    downtownfoursquare_dataset = pickle.load(fp)
# print(type(Scarborough_foursquare_dataset))
# Scarborough_foursquare_dataset



EOFError: Ran out of input

In [38]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [39]:
downtown_venues = get_venue_dataset(downtown_foursquare_dataset)

Number of Venuse in Coordination "M5A" Posal Code and "Harbourfront" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5A" Posal Code and "Regent Park" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5B" Posal Code and "Ryerson" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5B" Posal Code and "Garden District
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5C" Posal Code and "St. James Town" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5E" Posal Code and "Berczy Park" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5G" Posal Code and "Central Bay Street
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M6G" Posal Code and "Christie
" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5H" Posal Code and "Adelaide" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5H" Posal Code and "King" Negihborhood(s) is:
100
Number of Venuse in Coordination "M5H" Posal Code and "Richmond
" Negihborhood(

In [40]:

downtown_venues.head()



,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M5A,Harbourfront,43.65426,-79.360636,Roselle Desserts,This spot is popular,Bakery,143
1,M5A,Harbourfront,43.65426,-79.360636,Tandem Coffee,This spot is popular,Coffee Shop,122
2,M5A,Harbourfront,43.65426,-79.360636,Cooper Koo YMCA,This spot is popular,Gym / Fitness Center,247
3,M5A,Harbourfront,43.65426,-79.360636,Impact Kitchen,This spot is popular,Restaurant,376
4,M5A,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,This spot is popular,Breakfast Spot,54


In [41]:


downtown_venues.tail()





,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
2917,M4Y,Church and Wellesley,43.66586,-79.38316,Four Seasons Hotel Toronto,This spot is popular,Hotel,832
2918,M4Y,Church and Wellesley,43.66586,-79.38316,Jimmy's Coffee,This spot is popular,Coffee Shop,851
2919,M4Y,Church and Wellesley,43.66586,-79.38316,Trattoria Nervosa,This spot is popular,Italian Restaurant,858
2920,M4Y,Church and Wellesley,43.66586,-79.38316,Coffee Public,This spot is popular,Coffee Shop,617
2921,M4Y,Church and Wellesley,43.66586,-79.38316,Onnki Donburi,This spot is popular,Japanese Restaurant,448


In [42]:
downtown_venues.to_csv('downtown_venues.csv')


In [43]:
downtown_venues = pd.read_csv('downtown_venues.csv')

In [44]:
neigh_list = list(downtown_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Downtown:')
print(len(neigh_list))
print('List of Neighborhoods inside Downtown:')
neigh_list

Number of Neighborhoods inside Downtown:
36
List of Neighborhoods inside Downtown:


['Harbourfront',
 'Regent Park',
 'Ryerson',
 'Garden District\n',
 'St. James Town',
 'Berczy Park',
 'Central Bay Street\n',
 'Christie\n',
 'Adelaide',
 'King',
 'Richmond\n',
 'Harbourfront East\n',
 'Toronto Islands',
 'Union Station',
 'Design Exchange',
 'Toronto Dominion Centre',
 'Commerce Court',
 'Victoria Hotel\n',
 'Harbord\n',
 'University of Toronto',
 'Chinatown',
 'Grange Park',
 'Kensington Market',
 'CN Tower',
 'Bathurst Quay\n',
 'Island airport\n',
 'Harbourfront West\n',
 'King and Spadina',
 'Railway Lands',
 'South Niagara',
 'Rosedale',
 'Stn A PO Boxes 25 The Esplanade\n',
 'Cabbagetown',
 'First Canadian Place',
 'Underground city',
 'Church and Wellesley']

In [45]:
neigh_venue_summary = downtown_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Adelaide,100,100,100,100,100,100,100
Bathurst Quay\n,16,16,16,16,16,16,16
Berczy Park,100,100,100,100,100,100,100
CN Tower,16,16,16,16,16,16,16
Cabbagetown,44,44,44,44,44,44,44


In [46]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(downtown_venues['Venue Category'].unique())

There are 208 uniques categories.
Here is the list of different categories:


['Bakery',
 'Coffee Shop',
 'Gym / Fitness Center',
 'Restaurant',
 'Breakfast Spot',
 'Historic Site',
 'Farmers Market',
 'Chocolate Shop',
 'Park',
 'Spa',
 'Dessert Shop',
 'Mediterranean Restaurant',
 'Pub',
 'Italian Restaurant',
 'Mexican Restaurant',
 'Performing Arts Venue',
 'Liquor Store',
 'French Restaurant',
 'Café',
 'Thai Restaurant',
 'Tech Startup',
 'Greek Restaurant',
 'Shoe Store',
 'Theater',
 'Bar',
 'Animal Shelter',
 'Event Space',
 'Cosmetics Shop',
 'Brewery',
 'Ice Cream Shop',
 'Spanish Restaurant',
 'Pool',
 'Food Truck',
 'Art Gallery',
 'German Restaurant',
 'Pet Store',
 'Diner',
 'Middle Eastern Restaurant',
 'Asian Restaurant',
 'Grocery Store',
 'Furniture / Home Store',
 'Pizza Place',
 'Karaoke Bar',
 'Sushi Restaurant',
 'Gastropub',
 'Dance Studio',
 'Beer Store',
 'Bank',
 'Indian Restaurant',
 'Electronics Store',
 'Skating Rink',
 'Burger Joint',
 'Auto Dealership',
 'Pharmacy',
 'Discount Store',
 'Clothing Store',
 'Burrito Place',
 'Comic S

In [47]:
# Just for fun and deeper understanding
print(type(downtown_venues[['Venue Category']]))

print(type(downtown_venues['Venue Category']))



<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [48]:
# one hot encoding
downtown_onehot = pd.get_dummies(data = downtown_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
downtown_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,M5A,Harbourfront,43.65426,-79.360636,Roselle Desserts,This spot is popular,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M5A,Harbourfront,43.65426,-79.360636,Tandem Coffee,This spot is popular,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M5A,Harbou

In [49]:
# This list is created manually 
important_list_of_features = [
 

 'Neighborhood Latitude',
 'Neighborhood Longitude',


 'Art Gallery',
 'Art Museum',

 
 'Arts & Crafts Store',
 
 
 
 
 'Café',

]

In [50]:

downtown_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,M5A,Harbourfront,43.65426,-79.360636,Roselle Desserts,This spot is popular,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M5A,Harbourfront,43.65426,-79.360636,Tandem Coffee,This spot is popular,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M5A,Harbou

In [51]:
downtown_onehot = downtown_onehot.loc[:, ~downtown_onehot.columns.duplicated()]

In [52]:

downtown_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,M5A,Harbourfront,43.65426,-79.360636,Roselle Desserts,This spot is popular,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M5A,Harbourfront,43.65426,-79.360636,Tandem Coffee,This spot is popular,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M5A,Harbourfront,43.65426,-

visualize the clusters in map

In [53]:
downtown_onehot = downtown_onehot.drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()

downtown_onehot.head()


,Unnamed: 0,Distance,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Adelaide,84950,40595,0,0,0,0,4,0,0,1,1,1,2,0,0,0,0,2,0,3,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,7,0,0,0,0,0,0,1,0,4,0,0,0,0,0,3,0,2,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2,0,0,0,0,0,1,0,1,0,2,1,0,0,0,0,0,0,0,0,5,0,1,1,0,0,1,3,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2,1,0,4,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,2,0,0,2,1,0,0,0,0,0,2,4,0,0,1,0,1,1,0,0,0,0,0,0,1
Bathurst Quay\n,37176,12343,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
Berczy Park,5

In [58]:
feat_name_list = list(downtown_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
downtown_onehot['Total Restaurants'] = downtown_onehot[restaurant_list].sum(axis = 1)
downtown_onehot = downtown_onehot.drop(columns = restaurant_list)


feat_name_list = list(downtown_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
downtown_onehot['Total Joints'] = downtown_onehot[joint_list].sum(axis = 1)
downtown_onehot = downtown_onehot.drop(columns = joint_list)

examine each of the 5 clusters

In [59]:
downtown_onehot

,Unnamed: 0,Distance,Adult Boutique,Airport,Airport Lounge,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burrito Place,Butcher,Café,Candy Store,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Farm,Farmers Market,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Library,Liquor Store,Lounge,Martial Arts Dojo,Men's Store,Metro Station,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Record Shop,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Taco Place,Tailor Shop,Tea Room,Tech Startup,Theater,Track,Trail,Train Station,Tunnel,University,Video Game Store,Video Store,Wine Bar,Wine Shop,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Adelaide,84950,40595,0,0,0,0,0,1,1,1,0,0,0,2,0,3,0,0,0,0,0,0,1,1,0,0,1,1,0,7,0,0,0,0,1,0,4,0,0,0,0,3,0,2,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,2,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,5,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,2,0,0,1,0,0,0,4,0,0,1,0,1,0,0,0,0,1
Bathurst Quay\n,37176,12343,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
Berczy Park,54950,55537,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,1,1,1,0,1,0,1,0,0,0,0,0,7,0,2,0,1,1,3,7,0,0,0,0,1,0,2,1,0,0,1,0,0,0,1,0,0,0,0,0,2,0,0,0,1,1,0,0,0,3,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,6,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,1,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
CN Tower,36920,12343,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
Cabbagetown,112442,23400,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
Central Bay Street\n,64950,53054,0,0,0,0,0,2,1,2,0,0,0,0,0,2,0,0,0,1,0,0,1,2,0,0,4,1,0,5,0,0,0,0,2,0,7,0,0,0,1,2,0,1,1,0,0,0,1,0,1,2,0,0,

In [56]:


# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(downtown_onehot)



In [57]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = downtown_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Unnamed: 0,Distance,Adult Boutique,Airport,Airport Lounge,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burrito Place,Butcher,Café,Candy Store,Cheese Shop,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Theater,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Farm,Farmers Market,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Lake,Library,Liquor Store,Lounge,Martial Arts Dojo,Men's Store,Metro Station,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Record Shop,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Taco Place,Tailor Shop,Tea Room,Tech Startup,Theater,Track,Trail,Train Station,Tunnel,University,Video Game Store,Video Store,Wine Bar,Wine Shop,Yoga Studio,Total Restaurants,Total Joints,Total Sum
G1,269950.000000,39633.000000,2.500000e-01,0.000000,0.000000,0.000000e+00,0.0,0.750000,0.000000,0.250000,0.000000e+00,0.000000e+00,0.000000e+00,1.750000,0.0,1.250000,0.0,7.500000e-01,0.000000e+00,1.000000,0.000000e+00,2.500000e-01,1.000000,1.250000,0.000000,0.000000e+00,0.500000,0.750000,0.000000e+00,6.250000,0.000000e+00,5.000000e-01,0.000000e+00,2.500000e-01,2.500000e-01,0.750000,8.750000,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e-01,1.750000,0.000000e+00,0.250000,0.500000,0.000000e+00,0.500000,1.250000,0.500000,0.000000e+00,0.000000,0.750000,0.000000e+00,0.250000,0.500000,0.000000e+00,0.000000e+00,0.500000,0.0,0.000000e+00,0.500000,0.750000,2.500000e-01,0.250000,0.000000e+00,0.250000,2.250000,0.75,0.250000,0.000000e+00,0.000000e+00,0.0,0.500000,1.750000,0.750000,0.000000,2.500000e-01,0.000000e+00,0.250000,0.000000e+00,2.500000e-01,2.500000e-01,5.750000,0.500000,1.000000,0.000000e+00,0.000000,2.500000e-01,0.000000e+00,2.500000e-01,0.000000,0.000000e+00,0.000000e+00,2.500000e-01,0.250000,0.0,5.000000e-01,0.000000e+00,0.000000,0.500000,0.750000,7.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e-01,0.500000,0.750000,2.500000e-01,0.000000e+00,0.000000e+00,0.750000,0.000000e+00,0.000000,0.000000e+00,2.500000e-01,0.000000e+00,1.000000,0.000000e+00,0.500000,0.000000e+00,0.000000e+00,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e-01,1.250000e+00,2.500000e-01,0.25,0.000000,0.000000e+00,0.000000,0.000000e+00,0.750000,0.000000e+00,0.750000,0.000000e+00,0.0,0.000000,0.000000e+00,5.000000e-01,7.500000e-01,0.500000,2.000000,0.0,0.250000,0.000000,2.500000e-01,1.000000,0.000000e+00,1.750000,0.000000e+00,0.000000e+00,0.750000,0.000000,0.000000e+00,2.500000e-01,2.500000e-01,0.000000,2.500000e-01,0.250000,25.750000,2.750000,309682.500000
G5,199950.000000,47824.166667,5.000000e-01,0.000000,0.000000,-6.938894e-18,0.0,1.166667,0.000000,1.166667,-3.469447e-18,-6.938894e-18,8.333333e-01,4.333333,0.0,5.500000,0.0,1.666667e-01,-3.469447e-18,1.500000,-6

best group is G1, second group is g5, thirtd group is g3